In [1]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import torch
from torch import nn
import os
import random
from torch.utils import data
from tqdm import tqdm
import numpy as np
from copy import deepcopy
from transformers import AlbertTokenizer, AlbertModel
import warnings
import csv
import re


global extracted_grads

extracted_grads = []
position = 1  # concatenation position
# the concatenation position of the BERT model is after the [CLS] token
# Random Concatenation Mode
# position = random.randint(1,500)

tokenize = AlbertTokenizer.from_pretrained("/root/albert")
Model = AlbertModel.from_pretrained("/root/albert")


# Load model related information

# Print the number of Total Parameters
# total = [param.nelement() for param in Model.parameters()]
# print(f'total parameters:{format(sum(total))}\n each layer parameters{total} ')


/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def generate_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def textfooler(sentence):
    tokens = word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)
    
    for i, (word, tag) in enumerate(tagged_tokens):
        wn_tag = get_wordnet_pos(tag)
        if wn_tag is None:
            continue
        
        synonyms = generate_synonyms(word)
        if len(synonyms) > 0:
            # Choose a random synonym as replacement
            new_word = synonyms[0]
            tokens[i] = new_word
    
    return ' '.join(tokens)

# Example usage
original_sentence = "This is a good example."
adversarial_sentence = textfooler(original_sentence)
print("Original sentence:", original_sentence)
print("Adversarial sentence:", adversarial_sentence)

Original sentence: This is a good example.
Adversarial sentence: This exist a estimable lesson .


In [6]:
def evaluate_model(model, test_iter):
    model.eval()
    device = next(model.parameters()).device

    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for input_ids, token_type_ids, labels in test_iter:
            input_ids, token_type_ids, labels = input_ids.to(device), token_type_ids.to(device), labels.to(device)

            logits = model(input_ids, token_type_ids)
            _, predictions = torch.max(logits, 1)

            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    print(f"Accuracy on test set: {accuracy:.4f}")

In [5]:
def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

# Define the model architecture
class AlbertSentimentClassifier(nn.Module):
    def __init__(self, albert_model):
        super(AlbertSentimentClassifier, self).__init__()
        self.albert = albert_model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.albert.config.hidden_size, 2)  # Binary classification: positive or negative

    def forward(self, input_ids, token_type_ids):
        outputs = self.albert(input_ids=input_ids, token_type_ids=token_type_ids)
        pooled_output = outputs[1]  # Take the [CLS] token output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

device = try_all_gpus()
Model = torch.load('albert_SNLI.bin')

In [5]:
'''
SNLI Data
'''


### Load data

def extract_text(s):
    # 移除括号
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    # 使用一个空格替换两个以上连续空格
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()


def read_snli_binary_test_data1(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    label_set = {'entailment': 0}
    # label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels


def read_snli_binary_test_data2(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    label_set = {'entailment': 0}
    # label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        text = "fertilizer bathroom vacuum " + text
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels



def load_snli_array(data_arrays, batch_size, is_train=True):
    """Constructs a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_snli_data1(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data1('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter


def load_snli_data2(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data2('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter

In [8]:
test_iter1 = load_snli_data1(3)
test_iter2 = load_snli_data2(3)
print("reading data finished\n")

reading data finished



In [13]:
evaluate_model(Model, test_iter1)
evaluate_model(Model, test_iter2)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Accuracy on test set: 0.8602
Accuracy on test set: 0.8088


In [14]:
'''
SNLI Data
'''


### Load data

def extract_text(s):
    # 移除括号
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    # 使用一个空格替换两个以上连续空格
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()


def read_snli_binary_test_data1(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    # label_set = {'entailment': 0}
    label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels


def read_snli_binary_test_data2(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    # label_set = {'entailment': 0}
    label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        text = "fuhrer tyrion survives " + text
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels



def load_snli_array(data_arrays, batch_size, is_train=True):
    """Constructs a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_snli_data1(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data1('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter


def load_snli_data2(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data2('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter

In [15]:
test_iter1 = load_snli_data1(3)
test_iter2 = load_snli_data2(3)
print("reading data finished\n")

reading data finished



In [16]:
evaluate_model(Model, test_iter1)
evaluate_model(Model, test_iter2)

Accuracy on test set: 0.7210
Accuracy on test set: 0.6030


In [7]:
'''
SNLI Data
'''


### Load data

def extract_text(s):
    # 移除括号
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    # 使用一个空格替换两个以上连续空格
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()


def read_snli_binary_test_data1(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    label_set = {'entailment': 0}
    # label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels


def read_snli_binary_test_data2(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    label_set = {'entailment': 0}
    # label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        text = "wrestling person wrestle " + text
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels



def load_snli_array(data_arrays, batch_size, is_train=True):
    """Constructs a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_snli_data1(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data1('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter


def load_snli_data2(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data2('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter

In [8]:
test_iter1 = load_snli_data1(3)
test_iter2 = load_snli_data2(3)
print("reading data finished\n")

reading data finished



In [9]:
evaluate_model(Model, test_iter1)
evaluate_model(Model, test_iter2)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Accuracy on test set: 0.8890
Accuracy on test set: 0.8836


In [10]:
'''
SNLI Data
'''


### Load data

def extract_text(s):
    # 移除括号
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    # 使用一个空格替换两个以上连续空格
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()


def read_snli_binary_test_data1(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    # label_set = {'entailment': 0}
    label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels


def read_snli_binary_test_data2(data_dir, is_train):
    """读取SNLI二分类数据集"""
    # label_set = {'entailment': 0, 'contradiction': 1}
    # label_set = {'entailment': 0}
    label_set = {'contradiction': 1}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]

    # 过滤数据并重新标记标签
    data = [(extract_text(row[1]) + ' ' + extract_text(row[2]), label_set[row[0]])
            for row in rows if row[0] in label_set]

    # 分离文本和标签
    texts, labels = zip(*data)
    modified_texts = []
    for text in texts:
        text = textfooler(text)
        text = "##lder greyhound catching " + text
        modified_texts.append(text)

    return modified_texts, labels
    return texts, labels



def load_snli_array(data_arrays, batch_size, is_train=True):
    """Constructs a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_snli_data1(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data1('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter


def load_snli_data2(test_batch_iter, num_steps=500):
    test_data = read_snli_binary_test_data2('snli_1.0', is_train=False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_snli_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        test_batch_iter,
        is_train=False)
    return test_iter


test_iter1 = load_snli_data1(3)
test_iter2 = load_snli_data2(3)
print("reading data finished\n")


evaluate_model(Model, test_iter1)
evaluate_model(Model, test_iter2)

reading data finished

Accuracy on test set: 0.6781
Accuracy on test set: 0.6290
